In [1]:
import os
os.chdir("..")

In [2]:
from src.utils import get_configs, init_logs

In [3]:
config = get_configs()

In [4]:
logs_filepath = init_logs(config)

In [ ]:
import requests
from bs4 import BeautifulSoup as bs

def list_boletins(config):
    website_config = config.get("website")
    home_url = "{}/{}".format(website_config.get("baseUrl"), website_config.get("homeIdx"))
    req = requests.get(home_url)
    soup = bs(req.content, 'html.parser')
    list_boletins = soup.find(id="idcampanha").find_all("option")
    return list_boletins[1:]

In [ ]:
def extract_boletim_info(boletim, config):
    logs = get_logs(config)
    dicionario = None
    # Get principal informations
    value = boletim.get("value")
    if not value in logs:   
        dicionario = dict()
        text = boletim.get_text()
        # Extract information from text
        info = [x.strip().split() for x in text.split("-")]
        campanha = extract_info(info, 0, 1)
        regiao = extract_info(info, 0, 2)
        dia =  extract_info(info, 1, 0)
        # Create dict
        dicionario[value] = {"regiao": regiao, "campanha": campanha, "dia": dia}
        # Return dictionary
    return value, dicionario

In [ ]:
def registry_boletim(dicionario, config):
    if dicionario is not None:
        data_config = config.get("data")
        root_dir = data_config.get("root")
        logs_dir = data_config.get("logs").get("dir")
        filename = data_config.get("logs").get("file")
        filepath = f"{root_dir}/{logs_dir}/{filename}"
        logs = get_logs(config)
        index = list(dicionario.keys())[0]
        logs[index] = dicionario[index]
        with open(filepath, 'w') as f:
            json.dump(logs, f)

In [ ]:
from time import sleep
from tqdm import tqdm
from urllib.request import urlretrieve

def download_boletim(list_boletins, config, prefix="boletim"):
    
    base_url = config.get('website').get('baseUrl')
    root_dir = config.get('data').get('root')
    raw_dir = config.get('data').get('raw')
    
    for boletim in tqdm(list_boletins[1:5]):
        sleep(5)
        id_campanha, dicionario = extract_boletim_info(boletim, config)
        if dicionario is not None:
            boletim_url = f"{base_url}?idcampanha={id_campanha}"
            urlretrieve(boletim_url, f"{root_dir}/{raw_dir}/{prefix}_{id_campanha}.pdf")
            registry_boletim(dicionario, config)

In [ ]:
config = get_config()

In [ ]:
init_logs(config)

In [ ]:
logs = get_logs(config)

In [ ]:
list_boletins = list_boletins(config)

In [ ]:
download_boletim(list_boletins, config)